### Calculate $\delta R$ matrix from combinations (CUDA Version)

Steps are:
- Take $\phi$ and $\eta$ data arrays,starts and stops. 
- Get combinations indices in same manner as done earlier
- For each pair, calculate deltaR as $$\delta r = \sqrt{\delta \eta^{2} + \delta \phi^{2}}$$ where $$\delta \eta = \eta_{2} - \eta_{1}$$$$ \delta \phi = \phi_{2} - \phi_{1}$$
- Get the matrix $$\delta R = \{ \delta r_{ij}\} \quad \forall \: (i,j) \: \in \: \mathrm{combinations \: indices}$$


In [16]:
import numpy
import pycuda.driver as cuda
from pycuda.compiler import *
import pycuda.autoinit

In [17]:
# Adapted from gen-reco script by Jim

NUMEVENTS = 100      # exact number of events
AVENUMJETS = 10      # average number of jets per event
PHILOW = -numpy.pi   # bounding box of phi (azimuthal angle) and eta (~polar angle)
PHIHIGH = numpy.pi
ETALOW = -5
ETAHIGH = 5
ERRPHI = 0.01        # detector resolution
ERRETA = 0.01
RECOPROB = 0.95      # probability of not reconstructing a real jet
AVENUMFAKES = 1      # average number of spurious (fake) recontstructions

# simulate the generator-level jets
numgenjets = numpy.random.poisson(AVENUMJETS, NUMEVENTS).astype(numpy.int32)
genstops = numpy.cumsum(numgenjets).astype(numpy.int32)
genstarts = numpy.empty_like(genstops)
genstarts[0] = 0
genstarts[1:] = genstops[:-1]
genphi = numpy.random.uniform(PHILOW, PHIHIGH, genstops[-1]).astype(numpy.float32)
geneta = numpy.random.uniform(ETALOW, ETAHIGH, genstops[-1]).astype(numpy.float32)

# simulate mismeasurement (error in reconstructing phi and eta)
phiwitherr = genphi + numpy.random.normal(0, ERRPHI, genstops[-1]).astype(numpy.float32)
etawitherr = geneta + numpy.random.normal(0, ERRETA, genstops[-1]).astype(numpy.float32)

In [18]:
# CUDA source
mod = SourceModule('''
#include <cmath>        // Not needed, but added to check external includes in PyCUDA 
extern "C"{

__device__ float deltaeta(float eta1,float eta2)
{
    return eta1-eta2;       // It is simple subtraction.
}

__device__ float deltaphi(float phi1, float phi2)
{
    return phi1-phi2;      // Here, we need to consider the case that phi1-phi2 is constrained 
                           // between [-pi, pi].
}

__global__ void deltaR(float* deltar,float* eta1, float* eta2, float* phi1, float* phi2,int* length, int* start, int* stop, int* pairs_len,int* lengths)
{
    // 3 dimensional indices
    // We will use hypotf() builtin CUDA function for the calculation of deltaR. 
    // The indices are generated in same way as combinations solved earlier. This indices are passeed onto
    // eta and phi arrays.
    
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    int j = blockIdx.y*blockDim.y + threadIdx.y;
    int k = blockIdx.z*blockDim.z + threadIdx.z;
    if (i <length[0]){
        if(j<lengths[i] && k<lengths[i]){
            deltar[pairs_len[i] + j*lengths[i] + k] = hypotf(deltaeta(eta1[j+start[i]], eta2[k+start[i]]), deltaphi(phi1[j + start[i]], phi2[k+start[i]]));
        }
    }
}
}
''', no_extern_c = True)

C:\Users\Jaydeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu



In [19]:
# Counts per event
lengths = genstops-genstarts

In [20]:
# Cumulative number of possible combinations per event
pairs_len = numpy.zeros(len(lengths)+1, dtype=numpy.int32)
pairs_len[1:] = numpy.cumsum(lengths*lengths)

In [23]:
# Initialize deltaR matrix
deltar = numpy.empty(pairs_len[-1], dtype=numpy.float32)

In [24]:
# Number of events = base_len
base_len = numpy.array(len(lengths)).astype(numpy.int32)

In [25]:
func = mod.get_function("deltaR")

In [27]:
func(cuda.InOut(deltar),cuda.In(geneta), cuda.In(etawitherr), cuda.In(genphi), cuda.In(phiwitherr), 
    cuda.In(base_len),cuda.In(genstarts), cuda.In(genstops), cuda.In(pairs_len), cuda.In(lengths), block=(1,8,8),grid = (len(numgenjets), 20, 20))

In [28]:
# Print the values
for i in range(6):
    print("Event:{} \n {}\n".format(i, deltar[pairs_len[i]:pairs_len[i+1]]))

Event:0 
 [  2.90420502e-02   3.30648017e+00   5.50417757e+00   2.30947161e+00
   4.82416296e+00   3.81520844e+00   3.61634445e+00   5.38179207e+00
   2.73391247e+00   3.82217360e+00   5.87689686e+00   5.70874977e+00
   3.10721803e+00   3.66029191e+00   3.32296658e+00   3.15319151e-02
   8.26857853e+00   3.36374760e+00   1.56420887e+00   7.11191368e+00
   5.20556927e+00   7.52635241e+00   6.00820541e+00   5.21266079e+00
   8.25873089e+00   8.35784340e+00   1.68209112e+00   1.86734962e+00
   5.49079418e+00   8.28513527e+00   4.79250913e-03   5.07497025e+00
   9.80455971e+00   3.78526783e+00   3.64276671e+00   1.91509104e+00
   4.36629534e+00   3.86690426e+00   1.29181123e+00   4.43316817e-01
   8.56915474e+00   9.12041092e+00   2.32642221e+00   3.39593410e+00
   5.07129669e+00   9.56179947e-03   4.83170795e+00   5.10045528e+00
   1.84980965e+00   4.15849924e+00   4.33974600e+00   1.90501451e+00
   4.91325617e+00   5.09589911e+00   4.22825193e+00   4.71459532e+00
   4.85332251e+00   1.55